In [ ]:
import math
import itertools


import numpy as np
import networkx as nx

In [ ]:
def calc_lambda(A, addremove=1, p=False):
    """Calculate the current lambda_2 and K_lambda_2 and return them."""
    L = np.diag(A.sum(1)) - A
    lambdas, vectors = np.linalg.eigh(L)
    sort = lambdas.argsort()
    lambdas = lambdas[sort]
    vectors = vectors[:,sort]

    l2 = lambdas[1]
    f = vectors[:,1]
    # l2_multiplicity = np.count_nonzero(np.isclose(lambdas, l2))

    search = []
    for i, j in itertools.combinations(range(len(f)), 2):
        if abs(f[i] - f[j]) > 10e-5 and ((addremove == 1 and A[i][j] == 0) or (addremove == -1 and A[i][j] == 1)):
            search.append((addremove * (f[i] - f[j]) ** 2, (i, j)))
    
    K_l2, link = max(search)

    if p:
        print(f"({l2:.5f}, {K_l2:.5f})")
        print(f"L  = {np.array2string(L, prefix='L  = ')}")
        print(f"eval={np.array2string(lambdas)}")
        print(f"evec={np.array2string(vectors, prefix='evec=')}")
        print('-----------------------------------------------\n')

    return round(l2, 5), link, f


In [ ]:
# Generate a random graph.
num_nodes = 12
G = nx.connected_watts_strogatz_graph(num_nodes, max(int(math.sqrt(num_nodes)), 2), 0.5)
initial_l2 = nx.algebraic_connectivity(G)

# Setup
target_l2 = 6

In [ ]:
steps_est = 0
current_l2 = initial_l2
A = nx.to_numpy_array(G)

while abs(target_l2 - current_l2) > 0.1:
    steps_est += 1
    if current_l2 < target_l2:
        current_l2, link, f = calc_lambda(A, 1)
        A[link[0]][link[1]] = 1
        A[link[1]][link[0]] = 1
        operation = 'added'
    else:
        current_l2, link, f = calc_lambda(A, -1)
        A[link[0]][link[1]] = 0
        A[link[1]][link[0]] = 0
        operation = 'removed'

    print(f'Step: {steps_est:3d}')
    print(f'l2 = {current_l2:6.3f}, Link {operation}: {link}')
    print('=' * 20, '\n')




In [ ]:

print(f'>{1.456789:7.3f}')